In [2]:
# 얼굴 인식을 위한 Haar Cascade 파일 경로
# face_cascade_path = 'venv\\haarcascade_frontalface_default.xml'
# eye_cascade_path = 'venv\\haarcascade_eye.xml'

# face_cascade = cv2.CascadeClassifier(face_cascade_path)
# eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

### import

In [14]:
import win32clipboard
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from PIL import ImageGrab, Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import pyperclip
import os
import io
from io import BytesIO
import base64
import tkinter as tk
from tkinter import messagebox

### 리소스 정리

In [28]:
# 작업물 제목
work_name = 'the-worlds-best-engineer'
find_name = '역대급'

# 에피소드 범위 설정
start_episode = 99
end_episode = 100

# 기본 URL
base_url = "https://manga-za.net/the-worlds-best-engineer-%e0%b8%95%e0%b8%ad%e0%b8%99%e0%b8%97%e0%b8%b5%e0%b9%88"

In [3]:
# Chrome 웹 드라이버 실행
driver = webdriver.Chrome()

# 화면 크기 정보 가져오기
screen_width = driver.execute_script("return window.screen.width") * 2
screen_height = driver.execute_script("return window.screen.height") * 2

# 에피소드 URL 생성 함수
def get_episode_url(base_url, episode_num):
    return f"{base_url}-{episode_num}/"


In [4]:
# 작업 보고 웹사이트 로그인 함수
def login_to_report_site(url, user_id, user_pw):
    driver.execute_script("window.open('about:blank', '_blank');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(url)
    time.sleep(0.5)
    
    # 아이디 입력
    id_input = driver.find_element(By.XPATH, '//*[@id="frm"]/div[1]/input')
    id_input.send_keys(user_id)
    
    # 비밀번호 입력
    pw_input = driver.find_element(By.XPATH, '//*[@id="frm"]/div[2]/input')
    pw_input.send_keys(user_pw)
    
    # 로그인 버튼 클릭
    login_button = driver.find_element(By.XPATH, '//*[@id="btLogin"]')
    login_button.click()

def send_to_clipboard(clip_type, data):
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(clip_type, data)
    win32clipboard.CloseClipboard()

# 로그인 정보
login_url = 'https://icopg.kcopa.or.kr/login'
login_id = '-login_pw = '-
# 작업 보고 웹사이트 로그인
login_to_report_site(login_url, login_id, login_pw)


# 수동자료 등록창 진입
driver.maximize_window()
time.sleep(0.5)
menu_button = driver.find_element(By.XPATH, '//*[@id="left"]/aside/section/div/ul/li[3]/a/p')
menu_button.click()
menu2_button = driver.find_element(By.XPATH, '//*[@id="left"]/aside/section/div/ul/li[3]/ul/li')
menu2_button.click()
time.sleep(0.5)
menu3_button = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[3]/div[1]/div[2]/div/input[1]')
menu3_button.click()
time.sleep(0.5)
# 창조절
current_size = driver.get_window_size()
half_width = current_size['width'] // 2
height = current_size['height']
driver.set_window_size(half_width, height)

### 자동화 작업 실행

In [30]:
# 각 에피소드 URL에 대해 작업 수행
for episode_num in range(start_episode, end_episode + 1):
    # 작업 창 전환
    driver.switch_to.window(driver.window_handles[0])
    episode_url = get_episode_url(base_url, episode_num)

    # 페이지 로딩 시간을 5초로 제한
    driver.set_page_load_timeout(5)

    try:
        driver.get(episode_url)
    except TimeoutException:
        pass
        
    # 페이지 중간으로 스크롤
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    middle_height = scroll_height // 4
    driver.execute_script(f"window.scrollTo(0, {middle_height})")
    
    while True:
        window_width = screen_width // 2
        window_height = screen_height
        
        # 크롬 창 화면 캡처
        screenshot = ImageGrab.grab(bbox=(0, 0, window_width, window_height))
        screenshot_np = np.array(screenshot)

        # 자동인식 정확도를 위해 화면 아래 광고 공간 제외 
        crop_height = int(window_height * 9 / 10)
        screenshot_np = screenshot_np[:crop_height, :]
        
        # BGR 색상 공간으로 변환
        screenshot_bgr = cv2.cvtColor(screenshot_np, cv2.COLOR_RGB2BGR)
        
        # 살색 검출
        hsv = cv2.cvtColor(screenshot_bgr, cv2.COLOR_BGR2HSV)
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)
        skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
        # 살색 영역 확장
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
        skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)
        
        # 살색 영역의 좌표 찾기
        contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) > 0:
            c = max(contours, key=cv2.contourArea)
            (x, y, w, h) = cv2.boundingRect(c)
            
            # 살색 영역의 크기가 일정 이상인 경우 캡처
            if w * h > 4000:
                # 채증 전 사용자 확인
                user_input = messagebox.askquestion("확인", "캡처하시겠습니까?")
                if user_input == 'yes':
                    time.sleep(0.5)
                    left_half_screenshot = ImageGrab.grab(bbox=(0, 90, window_width, window_height - 90))
                    left_half_screenshot_np = np.array(left_half_screenshot)

                    # 이미지를 캡처보드에 저장
                    img = Image.fromarray(left_half_screenshot_np)
                    output = io.BytesIO()
                    img.save(output, format='BMP')
                    data = output.getvalue()[14:]  # BMP 헤더 제거

                    # 클립보드에 이미지 복사
                    send_to_clipboard(win32clipboard.CF_DIB, data)
                    # 보고 작업
                    # 보고 창 전환
                    driver.switch_to.window(driver.window_handles[1])
                    driver.execute_script("window.scrollTo(0, 0)")
                    # 입력
                    work_name_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[2]/td/input[1]')
                    work_name_input.send_keys(f'{work_name} {episode_num}')
                    episode_url_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[1]/td/input[1]')
                    episode_url_input.send_keys(episode_url)
                    episode_url_click = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[1]/td/input[2]')
                    episode_url_click.click()
                    time.sleep(1)
                    episode_url_ok = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div[2]/button[2]')
                    episode_url_ok.click()
                    original_name_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[3]/td/input[2]')
                    original_name_input.click()
                    time.sleep(0.5)

                    # iframe으로 전환
                    wait = WebDriverWait(driver, 10)
                    iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'body > div.layer-popup-box.on > div > div > iframe')))

                    # 팝업 창 내의 요소 선택 및 입력
                    popup_input = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div/div/div[1]/div[1]/input[1]')))
                    popup_input.send_keys(find_name)
                    time.sleep(2) 
                    popup_find = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/div[1]/div[2]/input')))
                    popup_find.click()
                    time.sleep(1) 
                    popup_click = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/table/tbody/tr[1]/td[1]/label/span')))
                    popup_click.click()
                    time.sleep(1) 
                    popup_click2 = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/div[3]/input[2]')))
                    popup_click2.click()
                    time.sleep(0.5) 

                    # 메인 프레임으로 다시 전환
                    driver.switch_to.default_content()

                    # 캡처보드 요소에 붙여넣기
                    clipboard_element = driver.find_element(By.XPATH, '//*[@id="clipboard1"]')
                    clipboard_element.send_keys(Keys.CONTROL, 'v') 

                    # 결과창 클릭
                    end_click = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[3]/input[2]')
                    end_click.click()
                    try:
                        alert_popup = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div.bootbox.modal.fade.bootbox-alert.in')))
                        confirm_button = alert_popup.find_element(By.XPATH, '/html/body/div[6]/div/div/div[2]/button')
                        confirm_button.click()
                    except TimeoutException:
                        print("알림 팝업이 나타나지 않았습니다.")
                    break

                else:
                    driver.execute_script("window.scrollBy(0, 1000)")
        else:
            driver.execute_script("window.scrollBy(0, 1000)")


# 파묘 프로젝트

In [15]:
# 작업물 제목
work_name_Exhuma = 'Exhuma'
find_name_Exhuma = '파묘'
# URL
Exhuma_url = "https://movie22hd.com/exhuma-2024/"

In [5]:
# Chrome 웹 드라이버 실행
driver = webdriver.Chrome()

# 화면 크기 정보 가져오기
screen_width = driver.execute_script("return window.screen.width") * 2
screen_height = driver.execute_script("return window.screen.height") * 2


In [7]:
# 작업 보고 웹사이트 로그인 함수
def login_to_report_site(url, user_id, user_pw):
    driver.execute_script("window.open('about:blank', '_blank');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(url)
    time.sleep(0.5)
    
    # 아이디 입력
    id_input = driver.find_element(By.XPATH, '//*[@id="frm"]/div[1]/input')
    id_input.send_keys(user_id)
    
    # 비밀번호 입력
    pw_input = driver.find_element(By.XPATH, '//*[@id="frm"]/div[2]/input')
    pw_input.send_keys(user_pw)
    
    # 로그인 버튼 클릭
    login_button = driver.find_element(By.XPATH, '//*[@id="btLogin"]')
    login_button.click()

# 로그인 정보
login_url = 'https://icopg.kcopa.or.kr/login'
login_id = '24thmon1'
login_pw = '24thmon1qwe!@#'

# 작업 보고 웹사이트 로그인
login_to_report_site(login_url, login_id, login_pw)


# 수동자료 등록창 진입
driver.maximize_window()
time.sleep(0.5)
menu_button = driver.find_element(By.XPATH, '//*[@id="left"]/aside/section/div/ul/li[3]/a/p')
menu_button.click()
menu2_button = driver.find_element(By.XPATH, '//*[@id="left"]/aside/section/div/ul/li[3]/ul/li')
menu2_button.click()
time.sleep(0.5)
menu3_button = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[3]/div[1]/div[2]/div/input[1]')
menu3_button.click()
time.sleep(0.5)
# 창조절
current_size = driver.get_window_size()
half_width = current_size['width'] // 2
height = current_size['height']
driver.set_window_size(half_width, height)

In [16]:
# 보고 창 전환
driver.switch_to.window(driver.window_handles[1])
driver.execute_script("window.scrollTo(0, 0)")
# 입력
work_name_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[2]/td/input[1]')
work_name_input.send_keys(work_name_Exhuma)
episode_url_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[1]/td/input[1]')
episode_url_input.send_keys(Exhuma_url)
episode_url_click = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[1]/td/input[2]')
episode_url_click.click()
time.sleep(1)
episode_url_ok = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div[2]/button[2]')
episode_url_ok.click()
original_name_input = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[2]/table/tbody/tr[3]/td/input[2]')
original_name_input.click()
time.sleep(0.5)

# iframe으로 전환
wait = WebDriverWait(driver, 10)
iframe = wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'body > div.layer-popup-box.on > div > div > iframe')))

# 팝업 창 내의 요소 선택 및 입력
popup_input = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div/div/div[1]/div[1]/input[1]')))
popup_input.send_keys(find_name_Exhuma)
time.sleep(2) 
popup_find = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/div[1]/div[2]/input')))
popup_find.click()
time.sleep(1) 
popup_click = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/table/tbody/tr[1]/td[1]/label/span')))
popup_click.click()
time.sleep(1) 
popup_click2 = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div/div[3]/input[2]')))
popup_click2.click()
time.sleep(0.5) 

# 메인 프레임으로 다시 전환
driver.switch_to.default_content()

# 캡처보드 요소에 붙여넣기
clipboard_element = driver.find_element(By.XPATH, '//*[@id="clipboard1"]')
clipboard_element.send_keys(Keys.CONTROL, 'v') 

# 결과창 클릭
end_click = driver.find_element(By.XPATH, '//*[@id="body"]/div/div/div[2]/div[3]/input[2]')
end_click.click()
alert_popup = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div.bootbox.modal.fade.bootbox-alert.in')))
confirm_button = alert_popup.find_element(By.XPATH, '/html/body/div[6]/div/div/div[2]/button')
confirm_button.click()

UnexpectedAlertPresentException: Alert Text: 중복된 URL이 존재합니다.
Message: unexpected alert open: {Alert text : 중복된 URL이 존재합니다.}
  (Session info: chrome=124.0.6367.158)
Stacktrace:
	GetHandleVerifier [0x00007FF7ED271522+60802]
	(No symbol) [0x00007FF7ED1EAC22]
	(No symbol) [0x00007FF7ED0A7CE4]
	(No symbol) [0x00007FF7ED13A986]
	(No symbol) [0x00007FF7ED11A923]
	(No symbol) [0x00007FF7ED0E8FEC]
	(No symbol) [0x00007FF7ED0E9C21]
	GetHandleVerifier [0x00007FF7ED5741BD+3217949]
	GetHandleVerifier [0x00007FF7ED5B6157+3488183]
	GetHandleVerifier [0x00007FF7ED5AF0DF+3459391]
	GetHandleVerifier [0x00007FF7ED32B8E6+823622]
	(No symbol) [0x00007FF7ED1F5FBF]
	(No symbol) [0x00007FF7ED1F0EE4]
	(No symbol) [0x00007FF7ED1F1072]
	(No symbol) [0x00007FF7ED1E18C4]
	BaseThreadInitThunk [0x00007FFA38A87344+20]
	RtlUserThreadStart [0x00007FFA39CE26B1+33]
